# 프로젝트: Vocabulary Size를 변경해서 시도해보기

In [1]:
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

#### 1) 데이터 확인

In [2]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

2113536/2110848 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [3]:
# 로드 데이터의 구성 확인
print('훈련 샘플의 수: {}'.format(len(x_train)))
print('테스트 샘플의 수: {}'.format(len(x_test)))

훈련 샘플의 수: 8982
테스트 샘플의 수: 2246


In [4]:
# 레이블 숫자 0부터 시작되므로 최댓값구한후 1더하면 현재 클래스 개수 확인 가능

num_classes = max(y_train) + 1
print('클래스의 수 : {}'.format(num_classes))

클래스의 수 : 46


#### 2) 데이터 복원

In [5]:
# 제공된 로이터 뉴스 데이터를 단어key key값으로, 고유한 정수를 value로 가지는 dictionary를 word_index로 저장

word_index = reuters.get_word_index(path="reuters_word_index.json")
print('=3')

557056/550378 [==============================] - 0s 0us/step
=3


In [6]:
# <로이터 뉴스 데이터 정의> word_index에 입력으로 했을 때, 얻는 숫자보다는 +3을 한 숫자가 원래 고유한 숫자
    
index_to_word = { index+3 : word for word, index in word_index.items() }
print('=3')

=3


In [7]:
#<로이터 뉴스 데이터 정의>  <pad>, <sos>, <unk> 자연어 처리를 위한 맵핑 번호

# index_to_word에 숫자 0은 <pad>, 숫자 1은 <sos>, 숫자 2는 <unk>를 넣어줍니다.
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token
print('=3')

=3


In [8]:
# index_to_word 를 통해 텍스트로 원래 복원하기

print(' '.join([index_to_word[index] for index in x_train[0]])) 

<sos> mcgrath rentcorp said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


In [9]:
# 이제 전체 훈련용 뉴스 데이터와 전체 테스트용 뉴스 데이터를 텍스트 데이터로 변환

decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

8982


In [10]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

2246


In [11]:
x_train[:1]

['<sos> mcgrath rentcorp said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3']

In [12]:
x_test[:1]

['<sos> the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to accommodate growth and expansion plans for waldbaum inc and shopwell inc over the next two years a and p said the acquisition of shopwell in august 1986 and waldbaum in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt tengelmann warenhandelsgesellschaft of west germany reuter 3']

#### 3) 벡터화 하기

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
print('=3')

=3


In [14]:
# DTM 생성 및 DTM 크기 확인

dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 26506)


In [15]:
# TF-IDF Matrix는 사이킷런의 TfidfTransformer()를 통해서 생성

tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 26506)


In [16]:
# 정확도 측정을 위해 테스트 데이터 전처리 필요, 이에 테스트 데이터 TF-IDF  행렬로 변환 필요

x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

## 다양한 머신러닝 모델 사용해보기

In [17]:
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

print('=3')

=3


### 1) Naive Bayes Classifier(NB)
- 스팸메일 예 (https://youtu.be/3JWLIV3NaoQ)
- P(coupon | spam) * P(spam)/ P(coupon) 
    - 총 8개의 메일 중 4개의 메일이 스팸 메일이므로 P(spam) = 4/8 = 1/2
    - 총 8개의 메일 중 3개의 메일이 coupon이라는 단어를 포함하므로 P(coupon) = 3/8
    - 총 4개의 스팸 메일 중 2개의 메일이 coupon이라는 단어를 포함하므로 P(coupon | spam) = 2/4 = 1/2
    - 정답 2/3 즉, 66.7%     P(coupon | spam) * P(spam)/ P(coupon) 

In [18]:
model = MultinomialNB()  # 나이브 베이즈 분류기
model.fit(tfidfv, y_train)

MultinomialNB()

In [19]:
predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측값을 얻어 정확도 측정
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.5997328584149599


In [20]:
# 예측 테스트

x_test[3]

'<sos> graham mccormick oil and gas partnership said it completed the sale of interests in two major oil and gas fields to lt energy assets international corp for 21 mln dlrs the company said it sold about one half of its 50 pct interest in the oak hill and north rucias fields its two largest producing properties it said it used about 20 mln dlrs of the proceeds to prepay principal on its senior secured notes semi annual principal payments on the remaining 40 mln dlrs of notes have been satisfied until december 1988 as a result it said the company said the note agreements were amended to reflect an easing of some financial covenants and an increase of interest to 13 5 pct from 13 0 pct until december 1990 it said the noteholders exercise price for 1 125 000 warrants was also reduced to 50 cts from 1 50 dlrs the company said energy assets agreed to share the costs of increasing production at the oak hill field reuter 3'

In [21]:
# 이 샘플의 레이블 확인

y_test[3]

4

#### F1-Score, Confusion Matrix

- 분류성능평가지표 
    - Precision(정밀도) : 정밀도란 모델이 True라고 분류한 것 중에서!! 실제 True인 것의 비율
    - Recall(재현율) :실제 True인 것 중에서!!(전체범위) 모델이 True라고 예측한 것의 비율
    - Accuracy(정확도) : 옮게 예측한 경우를 고려하는 지표
    - F1 score : Precision과 Recall의 조화평균, 데이터 label이 불균형 구조일 때, 모델의 성능을 정확하게 평가, 성능 하나의 숫자로 표현가능

In [22]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print('=3')

=3


In [23]:
# 재현율, F1점수를 구하는 classification_report() 함수를 제공

print(classification_report(y_test, model.predict(tfidfv_test), zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.79      0.21      0.33       105
           2       0.00      0.00      0.00        20
           3       0.72      0.92      0.81       813
           4       0.45      0.96      0.61       474
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00        14
           7       0.00      0.00      0.00         3
           8       0.00      0.00      0.00        38
           9       0.00      0.00      0.00        25
          10       0.00      0.00      0.00        30
          11       0.80      0.29      0.42        83
          12       0.00      0.00      0.00        13
          13       0.00      0.00      0.00        37
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         9
          16       0.75      0.18      0.29        99
          17       0.00    

- macro: 단순평균
- weighted avg: 각 클래스에 속하는 표본의 개수로 가중평균
- accuracy avg : 정확도. 전체 학습 데이터의 개수에서 클래스를 정확하게 맞춘 개수의 비율

#### 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) )
- 나이브 베이즈 분류기를 보완, 데이터의 불균형을 고려하여 가중치를 부여하는 특징이 있음

In [24]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [25]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7649154051647373


#### 3) 로지스틱 회귀(Logistic Regression)
- 소프트맥스(softmax) 함수를 사용한 다중 클래스 분류 알고리즘을 지원
    - 소프트맥스 함수는 클래스가 N개일 때, N차원의 벡터가 '각 클래스가 정답일 확률'을 표현하도록 정규화해주는 함수
    - 다중 클래스 분류를 위한 로지스틱 회귀를 소프트맥스 회귀(Softmax Regression), <mark><주의> 이름은 회귀지만, 실제로는 분류를 수행

In [26]:
# 소프트맥스 회귀는 LogisticRegression()을 통해서 구현
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [27]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.813446126447017


#### 4) 선형 서포트 벡터 머신(Linear Support Vector Machine)

- 일대다 방식은 각 클래스를 다른 모든 클래스와 구분하도록 이진 분류 모델을 학습

In [28]:
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [29]:
# 서포트 벡터 머신 사용하여 분류하기

predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.780053428317008


#### 5) 결정 트리(Decision Tree)
- 고차원, 희소한 데이터에는 성능이 나오지 않는 특징이 있다.
- 데이터를 전부 다 사용한다. 가장 좋은 질문이 나온다.
- 결정 트리는 훈련 데이터에 과적합(Overfitting)되는 경향

In [30]:
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [31]:
predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6211041852181657


#### 6) 랜덤 포레스트(Random Forest)
- 여러개의 작은 나무들로 구성된 숲과 같다.
- 중복된 데이터를 허용하므로서 모델의 편향이 올라간다.(오버피팅시 솔루션으로 사용된다)
- 랜덤이므로 가장 좋은 질문이 나오지 않을수 있다.
- 랜덤 포레스트는 이 문제를 앙상블로 해결. 가령 서로 다른 방향으로 과적합된 트리들을 조합하면 오히려 모델 전체에서 과적합을 피할 수 있다

In [32]:
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [33]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6544968833481746


#### 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 
- 그래디언트 부스팅 트리는 여러 개의 결정 트리를 묶어 만드는 앙상블 모델
- 그레디언트 부스팅은 랜덤 포레스트와 다르게 이전 트리의 오차를 보완하는 방식
- 특징: 일부 특성을 무시 보통 랜덤포레스트 사용후 예측시간면에서 만족스럽지 않을때 그래디언트 부스팅 트리를 시도해보는것이 좋다. 
- 장점: 예측 속도가 빠르다
- 단점: 훈련 시간의 속도가 좀 오래 걸리고, 트리 기반 모델의 특성으로 인해서 희소한 고차원 데이터에 대해서는 잘 동작하지 않는다

In [34]:
grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [35]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7702582368655387


#### 8) 앙상블기법중  보팅(Voting) 투표방법
- 하드 보팅 : 투표용지 1장, 결과물에 대한 최종값을 투표해서 결정
- 소프트 보팅 : 최종 결과물이 나올 확률값을 다 더해서 최종 결과물에 대한 각각의 확률을 구한 뒤 최종값을 도출

In [36]:
# 로지스틱 회귀, CNB, 그래디언트 부스팅 트리 모델을 사용하여 소프트 보팅을 하였을 때의 성능을 비교
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0))],
                 n_jobs=-1, voting='soft')

In [37]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8187889581478184


[num_words = none 단어 머신러닝 정확도 결과]
- 1) Naive Bayes Classifier(NB) 정확도 : 59%
- 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 76%
- 3) 로지스틱 회귀(Logistic Regression) 정확도: 81%
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 78%
- 5) 결정 트리(Decision Tree) 정확도: 62%
- 6) 랜덤 포레스트(Random Forest) 정확도: 65%
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 77%
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 81%

## 2. 빈도수 상위 5,000개의 단어만 사용

In [38]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=5000, test_split=0.2)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


#### 1) 데이터 복원하기

In [39]:
# 제공된 로이터 뉴스 데이터를 단어key key값으로, 고유한 정수를 value로 가지는 dictionary를 word_index로 저장

word_index = reuters.get_word_index(path="reuters_word_index.json")

In [40]:
# <로이터 뉴스 데이터 정의> word_index에 입력으로 했을 때, 얻는 숫자보다는 +3을 한 숫자가 원래 고유한 숫자
    
index_to_word = { index+3 : word for word, index in word_index.items() }
print('=3')

=3


In [41]:
#<로이터 뉴스 데이터 정의>  <pad>, <sos>, <unk> 자연어 처리를 위한 맵핑 번호

# index_to_word에 숫자 0은 <pad>, 숫자 1은 <sos>, 숫자 2는 <unk>를 넣어줍니다.
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token
print('=3')

=3


In [42]:
# index_to_word 를 통해 텍스트로 원래 복원하기

print(' '.join([index_to_word[index] for index in x_train[0]]))

<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


In [43]:
# 이제 전체 훈련용 뉴스 데이터와 전체 테스트용 뉴스 데이터를 텍스트 데이터로 변환

decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

8982


In [44]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

2246


In [45]:
x_train[:1]

['<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3']

In [46]:
x_test[:1]

['<sos> the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to <unk> growth and expansion plans for <unk> inc and <unk> inc over the next two years a and p said the acquisition of <unk> in august 1986 and <unk> in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt <unk> <unk> of west germany reuter 3']

#### 2) 벡터화 하기

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
print('=3')

=3


In [48]:
# DTM 생성 및 DTM 크기 확인

dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 4867)


In [49]:
# TF-IDF Matrix는 사이킷런의 TfidfTransformer()를 통해서 생성

tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 4867)


In [50]:
# 정확도 측정을 위해 테스트 데이터 전처리 필요, 이에 테스트 데이터 TF-IDF  행렬로 변환 필요

x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

## 다양한 머신러닝 모델 사용해보기

### 1) Naive Bayes Classifier(NB)

In [51]:
model = MultinomialNB()  # 나이브 베이즈 분류기
model.fit(tfidfv, y_train)

MultinomialNB()

In [52]:
predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측값을 얻어 정확도 측정
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6731967943009796


### 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) 

In [53]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [54]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7707034728406055


### 3) 로지스틱 회귀(Logistic Regression) 

In [55]:
# 소프트맥스 회귀는 LogisticRegression()을 통해서 구현
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [56]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8058771148708815


### 4) 선형 서포트 벡터 머신(Linear Support Vector Machine)

In [57]:
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [58]:
# 서포트 벡터 머신 사용하여 분류하기

predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7586821015138023


### 5) 의사 결정 트리(Decision Tree) 

In [59]:
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [60]:
predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6179875333926982


### 6) 랜덤 포레스트(Random Forest) 

In [61]:
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [62]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.701246660730187


### 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 

In [63]:
grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [64]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.767586821015138


### 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리)

In [65]:
# 로지스틱 회귀, CNB, 그래디언트 부스팅 트리 모델을 사용하여 소프트 보팅을 하였을 때의 성능을 비교
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0))],
                 n_jobs=-1, voting='soft')

In [66]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8161175422974176


[num_words = 5천개 단어 머신러닝 정확도 결과]
- 1) Naive Bayes Classifier(NB) 정확도 : 67%
- 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 77%
- 3) 로지스틱 회귀(Logistic Regression) 정확도: 80%
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 75%
- 5) 의사 결정 트리(Decision Tree) 정확도: 61%
- 6) 랜덤 포레스트(Random Forest) 정확도:  70%
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 76%
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 81%

## 3. 직접 단어 개수를 설정해서 사용 
- 나이브 베이즈 분류기, CNB, 로지스틱 회귀, 서포트 벡터 머신, 결정 트리, 랜덤 포레스트, 그래디언트 부스팅 트리, 보팅중 3개 이상

### 1)  n_words = 2000

In [67]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=2000, test_split=0.2)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


#### 1) 데이터 복원

In [68]:
# 제공된 로이터 뉴스 데이터를 단어key key값으로, 고유한 정수를 value로 가지는 dictionary를 word_index로 저장

word_index = reuters.get_word_index(path="reuters_word_index.json")

In [69]:
# <로이터 뉴스 데이터 정의> word_index에 입력으로 했을 때, 얻는 숫자보다는 +3을 한 숫자가 원래 고유한 숫자
    
index_to_word = { index+3 : word for word, index in word_index.items() }
print('=3')

=3


In [70]:
#<로이터 뉴스 데이터 정의>  <pad>, <sos>, <unk> 자연어 처리를 위한 맵핑 번호

# index_to_word에 숫자 0은 <pad>, 숫자 1은 <sos>, 숫자 2는 <unk>를 넣어줍니다.
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token
print('=3')

=3


In [71]:
# index_to_word 를 통해 텍스트로 원래 복원하기

print(' '.join([index_to_word[index] for index in x_train[0]]))

<sos> <unk> <unk> said as a result of its december acquisition of <unk> co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and <unk> operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


In [72]:
# 이제 전체 훈련용 뉴스 데이터와 전체 테스트용 뉴스 데이터를 텍스트 데이터로 변환

decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

8982


In [73]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

2246


In [74]:
x_train[:1]

['<sos> <unk> <unk> said as a result of its december acquisition of <unk> co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and <unk> operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3']

In [75]:
x_test[:1]

['<sos> the great <unk> and pacific <unk> co said its three year <unk> mln dlr capital program will be be substantially increased to <unk> growth and expansion plans for <unk> inc and <unk> inc over the next two years a and p said the acquisition of <unk> in august 1986 and <unk> in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations <unk> 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the <unk> capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt <unk> <unk> of west germany reuter 3']

#### 2) 벡터화 하기

In [76]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
print('=3')

=3


In [77]:
# DTM 생성 및 DTM 크기 확인

dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 1944)


In [78]:
# TF-IDF Matrix는 사이킷런의 TfidfTransformer()를 통해서 생성

tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 1944)


In [79]:
# 정확도 측정을 위해 테스트 데이터 전처리 필요, 이에 테스트 데이터 TF-IDF  행렬로 변환 필요

x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

### 다양한 머신러닝 모델 사용해보기

In [80]:
# 1) Naive Bayes Classifier(NB)
model = MultinomialNB()  # 나이브 베이즈 분류기
model.fit(tfidfv, y_train)

MultinomialNB()

In [81]:
predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측값을 얻어 정확도 측정
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6914514692787177


In [82]:
# 2) 컴플리먼트 나이브 베이즈 분류기
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [83]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7569011576135352


In [84]:
# 3) 로지스틱 회귀(Logistic Regression)
# 소프트맥스 회귀는 LogisticRegression()을 통해서 구현
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [85]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7813891362422084


In [86]:
# 4) 선형 서포트 벡터 머신(Linear Support Vector Machine)
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [87]:
# 서포트 벡터 머신 사용하여 분류하기

predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7297417631344613


In [88]:
# 5) 의사 결정 트리(Decision Tree)
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [89]:
predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6255565449688335


In [90]:
# 6) 랜덤 포레스트(Random Forest)
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [91]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6954585930543188


In [92]:
# 7) 그래디언트 부스팅 트리

grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [93]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.757346393588602


In [94]:
# 8) 앙상블 소프트 보팅(# 로지스틱 회귀, CNB, 그래디언트 부스팅 트리)
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0))],
                 n_jobs=-1, voting='soft')

In [95]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7978628673196795


[num_words = 2천개 단어 머신러닝 정확도 결과]
- 1) Naive Bayes Classifier(NB) 정확도 : 69%
- 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 75%
- 3) 로지스틱 회귀(Logistic Regression) 정확도: 78%
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 72%
- 5) 의사 결정 트리(Decision Tree) 정확도: 62%
- 6) 랜덤 포레스트(Random Forest) 정확도:  69%
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 75%
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 79%

### 2)  n_words = 4000

In [96]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=4000, test_split=0.2)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


#### 1) 데이터 복원

In [97]:
# 제공된 로이터 뉴스 데이터를 단어key key값으로, 고유한 정수를 value로 가지는 dictionary를 word_index로 저장

word_index = reuters.get_word_index(path="reuters_word_index.json")

In [98]:
# <로이터 뉴스 데이터 정의> word_index에 입력으로 했을 때, 얻는 숫자보다는 +3을 한 숫자가 원래 고유한 숫자
    
index_to_word = { index+3 : word for word, index in word_index.items() }
print('=3')

=3


In [99]:
#<로이터 뉴스 데이터 정의>  <pad>, <sos>, <unk> 자연어 처리를 위한 맵핑 번호

# index_to_word에 숫자 0은 <pad>, 숫자 1은 <sos>, 숫자 2는 <unk>를 넣어줍니다.
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token
print('=3')

=3


In [100]:
# index_to_word 를 통해 텍스트로 원래 복원하기

print(' '.join([index_to_word[index] for index in x_train[0]]))

<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and <unk> operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


In [101]:
# 이제 전체 훈련용 뉴스 데이터와 전체 테스트용 뉴스 데이터를 텍스트 데이터로 변환

decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

8982


In [102]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

2246


In [103]:
x_train[:1]

['<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and <unk> operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3']

In [104]:
x_test[:1]

['<sos> the great atlantic and pacific <unk> co said its three year 345 mln dlr capital program will be be substantially increased to <unk> growth and expansion plans for <unk> inc and <unk> inc over the next two years a and p said the acquisition of <unk> in august 1986 and <unk> in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt <unk> <unk> of west germany reuter 3']

#### 2) 벡터화 하기

In [105]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
print('=3')

=3


In [106]:
# DTM 생성 및 DTM 크기 확인

dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 3889)


In [107]:
# TF-IDF Matrix는 사이킷런의 TfidfTransformer()를 통해서 생성

tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 3889)


In [108]:
# 정확도 측정을 위해 테스트 데이터 전처리 필요, 이에 테스트 데이터 TF-IDF  행렬로 변환 필요

x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

### 다양한 머신러닝 모델 사용해보기

In [109]:
# 1) Naive Bayes Classifier(NB)
model = MultinomialNB()  # 나이브 베이즈 분류기
model.fit(tfidfv, y_train)

MultinomialNB()

In [110]:
predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측값을 얻어 정확도 측정
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6838824577025824


In [111]:
# 2) 컴플리먼트 나이브 베이즈 분류기
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [112]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7689225289403384


In [113]:
# 3) 로지스틱 회귀(Logistic Regression)
# 소프트맥스 회귀는 LogisticRegression()을 통해서 구현
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [114]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7978628673196795


In [115]:
# 4) 선형 서포트 벡터 머신(Linear Support Vector Machine)
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [116]:
# 서포트 벡터 머신 사용하여 분류하기

predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.757346393588602


In [117]:
# 5) 의사 결정 트리(Decision Tree)
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [118]:
predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6202137132680321


In [119]:
# 6) 랜덤 포레스트(Random Forest)
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [120]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6878895814781835


In [121]:
# 7) 그래디언트 부스팅 트리

grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [122]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7671415850400712


In [123]:
# 8) 앙상블 소프트 보팅(# 로지스틱 회귀, CNB, 그래디언트 부스팅 트리)
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0))],
                 n_jobs=-1, voting='soft')

In [124]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8130008904719501


[num_words = 4천개 단어 머신러닝 정확도 결과]
- 1) Naive Bayes Classifier(NB) 정확도 : 68%
- 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 76%
- 3) 로지스틱 회귀(Logistic Regression) 정확도: 79%
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 75%
- 5) 의사 결정 트리(Decision Tree) 정확도: 62%
- 6) 랜덤 포레스트(Random Forest) 정확도:  68%
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 76%
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 81%

### 3)  n_words = 7000

In [127]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=7000, test_split=0.2)

#### 1) 데이터 복원

In [128]:
# 제공된 로이터 뉴스 데이터를 단어key key값으로, 고유한 정수를 value로 가지는 dictionary를 word_index로 저장

word_index = reuters.get_word_index(path="reuters_word_index.json")

In [129]:
# <로이터 뉴스 데이터 정의> word_index에 입력으로 했을 때, 얻는 숫자보다는 +3을 한 숫자가 원래 고유한 숫자
    
index_to_word = { index+3 : word for word, index in word_index.items() }
print('=3')

=3


In [130]:
#<로이터 뉴스 데이터 정의>  <pad>, <sos>, <unk> 자연어 처리를 위한 맵핑 번호

# index_to_word에 숫자 0은 <pad>, 숫자 1은 <sos>, 숫자 2는 <unk>를 넣어줍니다.
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token
print('=3')

=3


In [131]:
# index_to_word 를 통해 텍스트로 원래 복원하기

print(' '.join([index_to_word[index] for index in x_train[0]]))

<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


In [132]:
# 이제 전체 훈련용 뉴스 데이터와 전체 테스트용 뉴스 데이터를 텍스트 데이터로 변환

decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print(len(x_train))

8982


In [133]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print(len(x_test))

2246


In [134]:
x_train[:1]

['<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3']

In [135]:
x_test[:1]

['<sos> the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to <unk> growth and expansion plans for <unk> inc and <unk> inc over the next two years a and p said the acquisition of <unk> in august 1986 and <unk> in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt <unk> <unk> of west germany reuter 3']

#### 2) 벡터화 하기

In [136]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
print('=3')

=3


In [137]:
# DTM 생성 및 DTM 크기 확인

dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 6807)


In [138]:
# TF-IDF Matrix는 사이킷런의 TfidfTransformer()를 통해서 생성

tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 6807)


In [139]:
# 정확도 측정을 위해 테스트 데이터 전처리 필요, 이에 테스트 데이터 TF-IDF  행렬로 변환 필요

x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

### 다양한 머신러닝 모델 사용해보기

In [140]:
# 1) Naive Bayes Classifier(NB)
model = MultinomialNB()  # 나이브 베이즈 분류기
model.fit(tfidfv, y_train)

MultinomialNB()

In [141]:
predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측값을 얻어 정확도 측정
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6647373107747105


In [142]:
# 2) 컴플리먼트 나이브 베이즈 분류기
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [143]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7684772929652716


In [144]:
# 3) 로지스틱 회귀(Logistic Regression)
# 소프트맥스 회귀는 LogisticRegression()을 통해서 구현
lr = LogisticRegression(C=10000, penalty='l2')  #학습이 과대적합 되는 것을 방지하고자 일종의 penalty를 부여 https://greatjoy.tistory.com/58
lr.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [145]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.807212822796082


In [146]:
# 4) 선형 서포트 벡터 머신(Linear Support Vector Machine)
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [147]:
# 서포트 벡터 머신 사용하여 분류하기

predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7689225289403384


In [148]:
# 5) 의사 결정 트리(Decision Tree)
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [149]:
predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6206589492430988


In [150]:
# 6) 랜덤 포레스트(Random Forest)
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [151]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6736420302760463


In [152]:
# 7) 그래디언트 부스팅 트리

grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [153]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7604630454140695


In [154]:
# 8) 앙상블 소프트 보팅(# 로지스틱 회귀, CNB, 그래디언트 부스팅 트리)
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0))],
                 n_jobs=-1, voting='soft')

In [155]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.815227070347284


[num_words = 7천개 단어 머신러닝 정확도 결과]
- 1) Naive Bayes Classifier(NB) 정확도 : 66%
- 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 76%
- 3) 로지스틱 회귀(Logistic Regression) 정확도: 80%
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 76%
- 5) 의사 결정 트리(Decision Tree) 정확도: 62%
- 6) 랜덤 포레스트(Random Forest) 정확도:  67%
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 76%
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 81%

## 4. 딥러닝 모델과 비교해보기
-  감정 분석 등에 사용했던 RNN이나 1-D CNN 등의 딥러닝 모델 중 하나를 선택해서 오늘 사용했던 데이터셋을 학습해 보고 나오는 결과를 비교

In [211]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [212]:
(x_train_deep, y_train_deep), (x_test_deep, y_test_deep) = reuters.load_data(num_words= None, test_split=0.2)

In [213]:
# 5. 케라스 패딩을 위한 도구  pad_sequences() 로 활용하기 

total_data_text = list(x_train_deep) + list(x_test_deep)
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 

print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)

#max_len의 인자로 정수를 주면, 해당 정수로 모든 문서의 길이를 동일하게 하며 값설젇이 전체 모델 성능에 영향

print('maxlen of pad_sequence:', maxlen)
print('전체 문장의 {} % 가 maxlen 설정값 이내에 포함합니다. \n'.format(np.sum(num_tokens < max_tokens)))

문장길이 평균 :  145.96419665122906
문장길이 최대 :  2376
문장길이 표준편차 :  145.8784764459447
maxlen of pad_sequence: 437
전체 문장의 10598 % 가 maxlen 설정값 이내에 포함합니다. 



In [214]:
# padding 방식 변경해 보기 -앞쪽('pre')  *RNN은 입력데이터가 순차적으로 처리 마지막 입력이 무의미, 'pre'가 훨씬 유리

x_train_deep = keras.preprocessing.sequence.pad_sequences(x_train_deep,padding = 'pre', maxlen = maxlen)
x_test_deep = keras.preprocessing.sequence.pad_sequences(x_test_deep, padding = 'pre', maxlen = maxlen)

####  RNN 모델 비교

<mark><b>[진행 순서]</b>

1. RNN 모델 설계
2. 훈련용 데이터셋 10000건을 분리하여 검증셋(validation set)으로 사용
3. 모델 학습
4. 모델 테스트셋으로 평가

In [215]:
# 1. RNN 모델 설계
vocab_size = 50000  # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 200  # 워드 벡터의 차원 수 (변경 가능한 하이퍼파라미터)

# model 설계 
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim))
model.add(keras.layers.LSTM(100))
model.add(keras.layers.Dense(46, activation = 'softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 200)         10000000  
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_7 (Dense)              (None, 46)                4646      
Total params: 10,125,046
Trainable params: 10,125,046
Non-trainable params: 0
_________________________________________________________________


In [216]:
# 2. 훈련용 데이터셋 2000건을 분리하여 검증셋(validation set)으로 사용

x_val = x_train_deep[:3000]
y_val = y_train_deep[:3000]

partial_x_train_deep = x_train_deep[3000:]
partial_y_train_deep = y_train_deep[3000:]

partial_y_train_deep = keras.utils.to_categorical(partial_y_train_deep)
y_val =  keras.utils.to_categorical(y_val)
y_test_deep = keras.utils.to_categorical(y_test_deep)

es = keras.callbacks.EarlyStopping(monitor= 'val_loss', mode = 'min', verbose = 1, patience = 2)

- 위와 같이 지정하면 validation set 의 loss 를 monitoring 한다는 뜻
- 만약 performance measure가 최소화 시켜야하는 것이면 mode를 min 으로, 최대화 시켜야하는 것이면 mode를 max로 지정한다. loss 의 경우, 최소화 시키는 방향으로 training 이 진행되므로 min 을 지정한다.
- validation accuracy 가 1% 증가하지 않는 경우, 성능의 증가가 없다고 정의한다. 

출처: https://3months.tistory.com/424 [Deep Play]

In [217]:
# 3. 모델 학습
model.compile(optimizer= 'adam', loss =  'categorical_crossentropy', metrics = ['accuracy'])

In [220]:
epochs = 10

history_deep = model.fit(partial_x_train_deep, partial_y_train_deep, epochs= epochs , 
                         batch_size = 32,
                         validation_data= (x_val,y_val),
                        verbose = 1)

Epoch 1/10
187/187 [==============================] - 27s 147ms/step - loss: 0.6815 - accuracy: 0.8245 - val_loss: 1.5611 - val_accuracy: 0.6463
Epoch 2/10
187/187 [==============================] - 25s 134ms/step - loss: 0.5233 - accuracy: 0.8679 - val_loss: 1.6269 - val_accuracy: 0.6433
Epoch 3/10
187/187 [==============================] - 21s 112ms/step - loss: 0.3981 - accuracy: 0.9061 - val_loss: 1.6712 - val_accuracy: 0.6570
Epoch 4/10
187/187 [==============================] - 21s 114ms/step - loss: 0.3139 - accuracy: 0.9258 - val_loss: 1.7108 - val_accuracy: 0.6520
Epoch 5/10
187/187 [==============================] - 21s 113ms/step - loss: 0.2360 - accuracy: 0.9465 - val_loss: 1.7545 - val_accuracy: 0.6667
Epoch 6/10
187/187 [==============================] - 21s 112ms/step - loss: 0.1908 - accuracy: 0.9537 - val_loss: 1.7891 - val_accuracy: 0.6663
Epoch 7/10
187/187 [==============================] - 25s 134ms/step - loss: 0.1565 - accuracy: 0.9592 - val_loss: 1.8131 - val_ac

In [221]:
# 4. 모델 테스트셋으로 평가
results = model.evaluate(x_test_deep,  y_test_deep, verbose=2)

print(results)

71/71 - 2s - loss: 2.0399 - accuracy: 0.6358
[2.0398776531219482, 0.6357969641685486]


- epochs = 5 : LSTM 적용 모델 63%의 정확도 확인할수 있었음. 의사결정 트리 정확도 62%와 랜덤 포레스트 정확도 65%의 중간 정도의 성능으로 전체적으로 보팅보다 떨어지는것을 확인 할수 있었다.
- epochs = 10 :  LSTM 적용 모델 63%의 정확도 확인할수 있었음. 의사결정 트리 정확도 62%와 랜덤 포레스트 정확도 65%의 중간 정도의 성능으로 전체적으로 보팅보다 떨어지는것을 확인 할수 있었다.

## 루브릭
- 3가지 단어 개수에 대해 8가지 머신러닝 기법을 적용하여 그중 최적의 솔루션을 도출하였다
- Vocabulary size에 따른 각 머신러닝 모델의 성능변화 추이를 살피고, 해당 머신러닝 알고리즘의 특성에 근거해 원인을 분석하였다.
- 동일한 데이터셋과 전처리 조건으로 딥러닝 모델의 성능과 비교하여 결과에 따른 원인을 분석하였다.

## 회고

[결론]
- 정확도는 8)보팅 > 3)로지스틱회귀 > 7)그래디언트 부스팅트리 > 2)CNB > 4)LSVM > 6) 랜덤포레스트> 1) NB> 5) 의사결정트리 의 순으로 성능이 좋은것으로 확인 되었다. 의사결정 트리가 고차원, 희소한 데이터에는 성능이 나오지 않는 특징이 있어서 그런것 같다. 

[확인 결과]
- 1) Naive Bayes Classifier(NB)는 num_words = none 일때  정확도 59%로 가장 낮고, 2천개 단어일때 69%로 높았다. 
- 2) 컴플리먼트 나이브 베이즈 분류기(CNB) )  num_words 와 상관 없이 정확도 75%~ 76%를 유지하였다.
- 3) 로지스틱 회귀(Logistic Regression)   num_words = 2천일때 정확도가 78%로 낮았고 대부분 78%~81%를 유지하였다.
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 는  num_words = 2천일때일때 정확도가 72%로 낮았고, num_words = none일때  78%로 정확도가 높았다.
- 5) 의사 결정 트리(Decision Tree)는 정확도가 61%~62%로 가장 성능이 좋지 않았다.
- 6) 랜덤 포레스트(Random Forest)는  num_words = none 일때  정확도 65%로 낮고, 5천개 단어일때 70%로 높았다. 
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 는 정확도가 대부분 75%~77%를 유지하였다.
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 는 정확도가  num_words = 2천일때 79%로 가장낮았고 대부분 81%의 정확도를 보였다. 


[num_words : 등장 빈도수 TEST 상세내용 ]

[num_words = none 단어 머신러닝 정확도 결과]
1) Naive Bayes Classifier(NB) 정확도 : 59%
2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 76%
3) 로지스틱 회귀(Logistic Regression) 정확도: 81%
4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 78%
5) 의사 결정 트리(Decision Tree) 정확도: 62%
6) 랜덤 포레스트(Random Forest) 정확도: 65%
7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 77%
8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 81%


[num_words = 5천개 단어 머신러닝 정확도 결과]
- 1) Naive Bayes Classifier(NB) 정확도 : 67%
- 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 77%
- 3) 로지스틱 회귀(Logistic Regression) 정확도: 80%
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 75%
- 5) 의사 결정 트리(Decision Tree) 정확도: 61%
- 6) 랜덤 포레스트(Random Forest) 정확도:  70%
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 76%
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 81%

[num_words = 2천개 단어 머신러닝 정확도 결과]
- 1) Naive Bayes Classifier(NB) 정확도 : 69%
- 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 75%
- 3) 로지스틱 회귀(Logistic Regression) 정확도: 78%
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 72%
- 5) 의사 결정 트리(Decision Tree) 정확도: 62%
- 6) 랜덤 포레스트(Random Forest) 정확도:  69%
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 75%
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 79%

[num_words = 4천개 단어 머신러닝 정확도 결과]
- 1) Naive Bayes Classifier(NB) 정확도 : 68%
- 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 76%
- 3) 로지스틱 회귀(Logistic Regression) 정확도: 79%
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 75%
- 5) 의사 결정 트리(Decision Tree) 정확도: 62%
- 6) 랜덤 포레스트(Random Forest) 정확도:  68%
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 76%
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 81%

[num_words = 7천개 단어 머신러닝 정확도 결과]
- 1) Naive Bayes Classifier(NB) 정확도 : 66%
- 2) 컴플리먼트 나이브 베이즈 분류기(Complement Naive Bayes Classifier(CNB) ) 정확도 : 76%
- 3) 로지스틱 회귀(Logistic Regression) 정확도: 80%
- 4) 선형 서포트 벡터 머신(Linear Support Vector Machine) 정확도 : 76%
- 5) 의사 결정 트리(Decision Tree) 정확도: 62%
- 6) 랜덤 포레스트(Random Forest) 정확도:  67%
- 7) 그래디언트 부스팅 트리(GradientBoostingClassifier) 정확도 : 76%
- 8) 앙상블 소프트 보팅(로지스틱 회귀, CNB, 그래디언트 부스팅 트리) 정확도 : 81%

[궁금한점] *간만에 창호퍼실님께 물어볼게 생겼는데 퇴실에 다른 퍼실님이 오셨다. 이렇게 또 타이밍이 어긋나는 ㅋㅋ

- 학습이 과적합 되는것을 방지하고자 일정의 패널티를 부여하고자 로지스틱회귀는 L2를 선형 서포트 벡터머신은  L1 규제항을 추가 하는데
개념이 타블로그( https://greatjoy.tistory.com/58)에서 봐도 좀 어려운감이 있다. 서로 달리 써도 되는건지도 궁금? 

[보완할점]
- 오래전 배워했던 RNN 딥러닝 모델이라 기억이 잘 나지 않아 반정도만 이해한것 같다.  추후 밑시딥2에서 다시 살펴봐야할것 같다. 

[개선요청]
- 3번 지문의 내용 개선 필요( 직접 단어 개수 설정해서 사용하되 사용할 아래 모델중 3가지 이상 실험하라의 뜻으로도 해석되는데 루브릭이 뒷부분에 나오기 때문에 사전에 자세하게 읽지 않으면 햇갈릴것 같습니다.
전 : 위 단계에서 5000으로 제시된 num_words를 다양하게 바꾸어 가며 성능을 확인해보세요. 변화된 단어 수에 따른 모델의 성능을 연구해 보세요. 최소 3가지 경우 이상을 실험해 보기를 권합니다.
후 : 최소 3가지 이상의 num_words 단어 개수 변경 실험을 진행해 보고 매 진행시 마다 아래의 8가지 모델 전부를 사용하시오) 